In [1]:
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")

In [2]:
housesPath =  "Houses.txt"
monthlyWaterConsumptionPath =  "MonthlyWaterConsumption.txt"
housesRDD =  sc.textFile(housesPath)
monthlyWaterConsumptionRDD =  sc.textFile(monthlyWaterConsumptionPath)

output1 = "outPart1"
output2 = "outPart2"

In [30]:
# Part1
# monthly 
# HID1,2022/01,66
def mapTrimester(l):
    id = str(l.split(",")[0])
    year = int(l.split(",")[1].split("/")[0])
    month = int(l.split(",")[1].split("/")[1])
    cons = float(l.split(",")[2])
    listOfN = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    listOfN[month-1] = cons
    return ((id, year), listOfN)

consPerTrim = monthlyWaterConsumptionRDD\
    .filter(lambda l: str(l.split(",")[1]).startswith("2022/") or str(l.split(",")[1]).startswith("2021/"))\
    .map(mapTrimester)\
    .reduceByKey(lambda l1, l2: [x + y for x, y in zip(l1, l2)])\
    .mapValues(lambda l: [sum(l[i:i+3]) for i in range(0, 12, 3)])\
    .map(lambda elem : (elem[0][0], (elem[0][1], elem[1])))\
    .cache()

In [31]:
consPerTrim.collect()

[('HID1', (2022, [269.0, 304.0, 397.0, 224.0])),
 ('HID2', (2021, [360.0, 252.0, 380.0, 399.0])),
 ('HID3', (2022, [284.0, 332.0, 330.0, 266.0])),
 ('HID4', (2022, [305.0, 231.0, 343.0, 280.0])),
 ('HID5', (2021, [350.0, 185.0, 269.0, 325.0])),
 ('HID6', (2021, [219.0, 190.0, 239.0, 345.0])),
 ('HID7', (2021, [310.0, 285.0, 293.0, 227.0])),
 ('HID8', (2021, [316.0, 149.0, 229.0, 262.0])),
 ('HID9', (2021, [324.0, 323.0, 148.0, 251.0])),
 ('HID10', (2021, [285.0, 339.0, 279.0, 318.0])),
 ('HID11', (2022, [341.0, 355.0, 371.0, 289.0])),
 ('HID12', (2022, [250.0, 275.0, 377.0, 396.0])),
 ('HID13', (2022, [301.0, 410.0, 267.0, 300.0])),
 ('HID14', (2022, [195.0, 274.0, 278.0, 346.0])),
 ('HID15', (2021, [212.0, 202.0, 291.0, 199.0])),
 ('HID16', (2021, [390.0, 372.0, 339.0, 393.0])),
 ('HID17', (2021, [282.0, 288.0, 329.0, 343.0])),
 ('HID18', (2022, [180.0, 329.0, 319.0, 334.0])),
 ('HID19', (2022, [246.0, 279.0, 415.0, 316.0])),
 ('HID20', (2021, [359.0, 237.0, 322.0, 322.0])),
 ('HID21'

In [35]:
def reduceCons(v1, v2):
    if(v1[0] == 2021):
        v = [x - y for x, y in zip(v2[1], v1[1])]
        return (2022, v)
    else:
        v = [x - y for x, y in zip(v1[1], v2[1])]
        return (2022, v)
def mapCons(v):
    sum = 0
    for i in v[1]:
        if i > 0:
            sum = sum + 1
    return sum
    
housesIncreasingCons = consPerTrim\
                    .reduceByKey(reduceCons)\
                    .mapValues(mapCons)\
                    .filter(lambda h: h[1] >= 3)

In [36]:
housesIncreasingCons.collect()

[('HID11', 3),
 ('HID14', 3),
 ('HID21', 3),
 ('HID22', 3),
 ('HID23', 3),
 ('HID24', 3),
 ('HID26', 4),
 ('HID28', 3),
 ('HID31', 3),
 ('HID41', 3),
 ('HID44', 3),
 ('HID53', 3),
 ('HID60', 4),
 ('HID63', 4),
 ('HID66', 3),
 ('HID88', 3),
 ('HID96', 3),
 ('HID100', 3),
 ('HID101', 3),
 ('HID117', 3),
 ('HID126', 3),
 ('HID137', 3),
 ('HID139', 3),
 ('HID145', 3),
 ('HID149', 4),
 ('HID152', 4),
 ('HID156', 3),
 ('HID166', 3),
 ('HID170', 4),
 ('HID181', 4),
 ('HID188', 3),
 ('HID190', 4),
 ('HID192', 3),
 ('HID195', 4),
 ('HID197', 3),
 ('HID200', 3),
 ('HID205', 3),
 ('HID209', 3),
 ('HID211', 4),
 ('HID214', 3),
 ('HID216', 4),
 ('HID224', 3),
 ('HID236', 3),
 ('HID245', 3),
 ('HID246', 3),
 ('HID252', 3),
 ('HID255', 3),
 ('HID264', 3),
 ('HID268', 3),
 ('HID286', 3),
 ('HID297', 4),
 ('HID298', 3),
 ('HID300', 3),
 ('HID303', 3),
 ('HID308', 3),
 ('HID309', 4),
 ('HID311', 4),
 ('HID321', 3),
 ('HID324', 3),
 ('HID325', 3),
 ('HID330', 3),
 ('HID333', 3),
 ('HID342', 3),
 ('HID350